In [1]:
# load library

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import time
import statistics
import pickle
import functools


from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, Constraint)
from ema_workbench import load_results
from ema_workbench.analysis import prim, dimensional_stacking, cart
from ema_workbench.util import ema_logging, utilities

from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench.em_framework import sample_uncertainties, MonteCarloSampler
ema_logging.log_to_stderr(ema_logging.INFO)

C:\Users\na0e2\Miniconda3\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)


<Logger EMA (DEBUG)>

In [2]:
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation

results = utilities.load_results('results/base_case_75.csv')

experiments, outcomes = results
outcomes = pd.DataFrame(outcomes)
experiments = pd.DataFrame(experiments)

outcome_total = pd.read_csv('results/75policies_with_total_values.csv')


results = experiments.join(outcome_total)
results = results.drop(columns="model")
results

[MainProcess/INFO] results loaded succesfully from C:\Users\na0e2\Documents\EPA1361\epa1361_open-master\final assignment\results\base_case_75.csv


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2,Total Expected Annual Damage,Total Expected Number of Deaths,Total Dike Investment Costs,Total RfR Total Costs,Total Expected Evacuation Costs
0,22.0,316.720229,1.5,0.508918,179.556360,1.0,0.056675,161.786225,1.0,0.411000,...,0.0,9.630148e+07,0.0,492600000.0,0.000000,1.704003e+07,0.002139,5.571625e+08,1.069800e+09,737.296215
1,37.0,48.449390,1.5,0.726517,71.388513,1.0,0.316716,140.869331,10.0,0.288527,...,0.0,9.630148e+07,0.0,492600000.0,0.000000,2.788806e+07,0.004377,5.571625e+08,1.069800e+09,1125.312347
2,116.0,342.466577,1.5,0.696968,164.240184,10.0,0.070253,241.641985,10.0,0.701739,...,0.0,9.630148e+07,0.0,492600000.0,0.000000,3.106400e+06,0.000434,5.571625e+08,1.069800e+09,240.005210
3,131.0,317.750054,10.0,0.514734,41.084696,10.0,0.670645,172.060501,1.5,0.002929,...,0.0,9.630148e+07,0.0,492600000.0,568.536776,6.589090e+08,0.125879,5.571625e+08,1.069800e+09,38728.384144
4,69.0,134.031006,1.0,0.056991,141.114722,10.0,0.138095,275.399627,1.0,0.807559,...,0.0,9.630148e+07,0.0,492600000.0,0.000000,1.957596e+07,0.003175,5.571625e+08,1.069800e+09,2151.356713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,23.0,151.955766,10.0,0.943525,149.089915,1.5,0.594206,270.182770,1.5,0.089715,...,0.0,5.525012e+07,0.0,679700000.0,0.000000,1.040420e+08,0.019924,7.619611e+08,1.474700e+09,5788.122696
74996,39.0,224.496227,1.0,0.390712,243.949795,1.0,0.090980,310.542851,1.5,0.569246,...,0.0,5.525012e+07,0.0,679700000.0,0.000000,8.310638e+07,0.008613,7.619611e+08,1.474700e+09,5271.800704
74997,45.0,347.284444,1.0,0.687342,59.820104,1.0,0.659417,229.083131,1.0,0.643934,...,0.0,5.525012e+07,0.0,679700000.0,0.000000,3.193923e+06,0.000458,7.619611e+08,1.474700e+09,244.787029
74998,124.0,106.585076,1.0,0.434781,132.822331,10.0,0.266128,170.757239,10.0,0.151776,...,0.0,5.525012e+07,0.0,679700000.0,0.000000,5.805561e+07,0.007140,7.619611e+08,1.474700e+09,2567.922822


In [3]:
# minimise the worst outcome and minimise the standard deviation
#defined as the median value multiplied by the interquartile distance plus one

def robustness(result):
    mean = np.median(result)
    iqr = sp.stats.iqr(result) + 1 #to aviod the return being 0
    robust = mean * iqr
    return robust

def total_robustness(*result):

    total_robust = robustness(sum(result))

    return total_robust


In [4]:
robustness_score = pd.DataFrame(columns = ["policy", "damage", "death", "dike_invest", "rfr_cost", "evacuation"])

for p in results["policy"].unique():

    index = results[results["policy"] == p].index
    
    damage = robustness(results["Total Expected Annual Damage"][index])
    death = robustness(results["Total Expected Number of Deaths"][index])
    dike_invest = robustness(results["Total Dike Investment Costs"][index])
    rfr_cost = robustness(results["Total RfR Total Costs"][index])
    evacuation = robustness(results["Total Expected Evacuation Costs"][index])
    
        
    policy_line = [p, damage, death, dike_invest, rfr_cost, evacuation]
    
    robustness_score = robustness_score.append(pd.Series(policy_line, index=["policy", "damage", "death", "dike_invest", "rfr_cost", "evacuation"]), ignore_index=True) 
robustness_score


,policy,damage,death,dike_invest,rfr_cost,evacuation
0,2.0,1.807322e+14,0.001541,5.571625e+08,1.069800e+09,7.783437e+05
1,3.0,2.448550e+13,0.000573,7.271277e+08,9.223000e+08,2.260960e+04
2,4.0,8.376979e+14,0.001775,5.168182e+08,3.638000e+08,5.959632e+06
3,5.0,5.409137e+13,0.000532,6.651736e+08,4.789000e+08,1.541017e+05
4,6.0,1.137168e+14,0.001044,7.372097e+08,1.012800e+09,3.022766e+05
...,...,...,...,...,...,...
70,72.0,1.629234e+16,0.008283,4.739753e+08,9.126000e+08,2.139238e+07
71,73.0,4.307115e+14,0.011271,6.030743e+08,1.093800e+09,0.000000e+00
72,74.0,1.423033e+15,0.045280,5.915852e+08,9.282000e+08,0.000000e+00
73,75.0,1.597251e+14,0.001531,9.011480e+08,1.033800e+09,2.995072e+05


In [5]:
dike_lists = ['A.1', 'A.2', 'A.3', 'A.4', 'A.5']

damage_column=[]
death_column=[]
investment_column=[]
rfr_column = []
evacuation_column = []

for c in outcomes.columns:
    if "Expected Annual Damage" in c:
        damage_column.append(c)

for c in outcomes.columns:
    if "Expected Number of Deaths" in c:
        death_column.append(c)
        
for c in outcomes.columns:
    if "Dike Investment Costs" in c:
        investment_column.append(c)
        
for c in outcomes.columns:
    if "RfR Total Costs" in c:
        rfr_column.append(c)
        
for c in outcomes.columns:
    if "Expected Evacuation Costs" in c:
        evacuation_column.append(c)


In [6]:
dike_model, planning_steps = get_model_for_problem_formulation(5)

robust_metrics = [
    ScalarOutcome('Damage', variable_name=damage_column,
                  function=total_robustness, kind=ScalarOutcome.MINIMIZE, expected_range=(0, 2e16)),
    ScalarOutcome('Deaths', variable_name=death_column,
                  function=total_robustness, kind=ScalarOutcome.MINIMIZE, expected_range=(0, 1)),
    ScalarOutcome('Dike Invest', variable_name=investment_column,
                  function=total_robustness, kind=ScalarOutcome.MINIMIZE, expected_range=(1e8, 1e9)),
    ScalarOutcome('RfR Cost', variable_name=rfr_column,
                  function=total_robustness, kind=ScalarOutcome.MINIMIZE, expected_range=(3e8, 2e9)),
    ScalarOutcome('Evacuation', variable_name=evacuation_column,
                  function=total_robustness, kind=ScalarOutcome.MINIMIZE, expected_range=(0, 5e7)),
]

#convergence_metrics = [HyperVolume.from_outcomes(dike_model.outcomes),
#                       EpsilonProgress()]

In [9]:
## from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.em_framework.evaluators import BaseEvaluator

BaseEvaluator.reporting_frequency = 0.1

# Record the run time
tic = time.time()

# Run MORO
with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robust_metrics,
                                                     scenarios=15,
                                                     nfe=200,
                                                     epsilons=[0.05,]*len(robust_metrics),
                                                     convergence=[EpsilonProgress()],
                                                     convergence_freq=1
                                                     )

toc = time.time()
print('Total run time:{} min'.format((toc - tic)/60))

# Save the run results
with open('results/initial_MORO.pkl', 'wb') as file_pi:
    pickle.dump((results, convergence), file_pi)

[MainProcess/INFO] generation 0: 0/100 nfe
[MainProcess/INFO] optimization completed, found 20 solutions


Total run time:1.2049282590548198 min


In [86]:
results

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Damage,Deaths,Dike Invest,RfR Cost,Evacuation
0,0,0,0,1,0,0,0,1,0,1,...,1,4,0,2,8,2.050453e+14,0.007340,6.090389e+08,4.909000e+08,0.000000e+00
1,0,1,1,0,0,0,0,0,0,1,...,4,0,0,9,0,2.063154e+15,0.009607,4.212896e+08,1.179900e+09,1.761991e+06
2,0,1,1,0,1,0,1,1,1,0,...,6,10,2,8,8,3.176041e+14,0.001910,6.845104e+08,4.791000e+08,6.926800e+06
3,0,0,1,0,0,0,1,0,0,1,...,7,0,5,5,5,1.676906e+15,0.047707,7.919648e+08,2.365000e+08,0.000000e+00
4,1,0,1,0,0,0,0,1,1,0,...,1,9,4,7,4,5.590446e+13,0.000852,7.117932e+08,6.079000e+08,2.130873e+05
5,0,0,0,0,0,0,1,1,0,0,...,10,3,6,9,0,0.000000e+00,0.000000,7.395451e+08,3.175000e+08,0.000000e+00
6,1,0,1,1,0,0,0,1,1,1,...,2,5,8,0,2,0.000000e+00,0.000000,4.551794e+08,6.908000e+08,0.000000e+00
7,1,1,0,1,1,0,1,0,1,1,...,6,1,2,0,5,5.357652e+15,0.006106,3.793910e+08,1.164700e+09,1.078584e+07
8,1,1,0,0,0,0,1,1,0,0,...,4,8,1,6,0,1.099688e+15,0.021350,5.150281e+08,4.730000e+08,0.000000e+00
9,0,0,1,1,1,1,0,1,1,1,...,2,2,2,5,9,1.125616e+15,0.027426,4.354306e+08,1.419100e+09,0.000000e+00


In [93]:
results.iloc[:, 0:-5].T.to_dict()

policy = []

for i in range(len(results)):
    pol = Policy(name="Policy " + str(i), **results.iloc[i, :-5].to_dict())
    policy.append(pol)

policy

[Policy({'0_RfR 0': 0.0, '0_RfR 1': 0.0, '0_RfR 2': 0.0, '1_RfR 0': 1.0, '1_RfR 1': 0.0, '1_RfR 2': 0.0, '2_RfR 0': 0.0, '2_RfR 1': 1.0, '2_RfR 2': 0.0, '3_RfR 0': 1.0, '3_RfR 1': 1.0, '3_RfR 2': 0.0, '4_RfR 0': 0.0, '4_RfR 1': 0.0, '4_RfR 2': 0.0, 'EWS_DaysToThreat': 0.0, 'A.1_DikeIncrease 0': 9.0, 'A.1_DikeIncrease 1': 4.0, 'A.1_DikeIncrease 2': 0.0, 'A.2_DikeIncrease 0': 10.0, 'A.2_DikeIncrease 1': 4.0, 'A.2_DikeIncrease 2': 4.0, 'A.3_DikeIncrease 0': 6.0, 'A.3_DikeIncrease 1': 2.0, 'A.3_DikeIncrease 2': 1.0, 'A.4_DikeIncrease 0': 7.0, 'A.4_DikeIncrease 1': 1.0, 'A.4_DikeIncrease 2': 4.0, 'A.5_DikeIncrease 0': 0.0, 'A.5_DikeIncrease 1': 2.0, 'A.5_DikeIncrease 2': 8.0}),
 Policy({'0_RfR 0': 0.0, '0_RfR 1': 1.0, '0_RfR 2': 1.0, '1_RfR 0': 0.0, '1_RfR 1': 0.0, '1_RfR 2': 0.0, '2_RfR 0': 0.0, '2_RfR 1': 0.0, '2_RfR 2': 0.0, '3_RfR 0': 1.0, '3_RfR 1': 0.0, '3_RfR 2': 1.0, '4_RfR 0': 1.0, '4_RfR 1': 1.0, '4_RfR 2': 1.0, 'EWS_DaysToThreat': 2.0, 'A.1_DikeIncrease 0': 0.0, 'A.1_DikeIncrease

In [129]:
tic = time.time()

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=policy)

toc = time.time()
print('Total run time:{} min'.format((toc - tic)/60))

# Write the results so this step can be skipped when doing multiple analyzes
with open('results/initial_MORO_2.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 50 scenarios * 13 policies * 1 model(s) = 650 experiments
[MainProcess/INFO] 65 cases completed
[MainProcess/INFO] 130 cases completed
[MainProcess/INFO] 195 cases completed
[MainProcess/INFO] 260 cases completed
[MainProcess/INFO] 325 cases completed
[MainProcess/INFO] 390 cases completed
[MainProcess/INFO] 455 cases completed
[MainProcess/INFO] 520 cases completed
[MainProcess/INFO] 585 cases completed
[MainProcess/INFO] 650 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


Total run time:6.370701336860657 min


In [130]:
experiments, outcomes = results
outcomes = pd.DataFrame(outcomes)
experiments = pd.DataFrame(experiments)
new_results = experiments.join(outcome_total)
new_results = new_results.drop(columns="model")

#total demage calculate
damage_column=[]
death_column=[]
investment_column=[]
rfr_column = []
evacuation_column = []

for c in new_results.columns:
    if "Expected Annual Damage" in c:
        damage_column.append(c)

for c in new_results.columns:
    if "Expected Number of Deaths" in c:
        death_column.append(c)
        
for c in new_results.columns:
    if "Dike Investment Costs" in c:
        investment_column.append(c)
        
for c in new_results.columns:
    if "RfR Total Costs" in c:
        rfr_column.append(c)
        
for c in new_results.columns:
    if "Expected Evacuation Costs" in c:
        evacuation_column.append(c)

new_results["Total Expected Annual Damage"] = new_results[damage_column].sum(axis=1)
new_results["Total Expected Number of Deaths"] = new_results[death_column].sum(axis=1)
new_results["Total Dike Investment Costs"] = new_results[investment_column].sum(axis=1)
new_results["Total RfR Total Costs"] = new_results[rfr_column].sum(axis=1)
new_results["Total Expected Evacuation Costs"] = new_results[evacuation_column].sum(axis=1)

In [176]:
robustness_score = pd.DataFrame(columns = ["policy", "damage", "death", "dike_invest", "rfr_cost", "evacuation"])

for p in new_results["policy"].unique():

    index = new_results[new_results["policy"] == p].index
    
    damage = robustness(new_results["Total Expected Annual Damage"][index])
    death = robustness(new_results["Total Expected Number of Deaths"][index])
    dike_invest = robustness(new_results["Total Dike Investment Costs"][index])
    rfr_cost = robustness(new_results["Total RfR Total Costs"][index])
    evacuation = robustness(new_results["Total Expected Evacuation Costs"][index])
    
        
    policy_line = [p, damage, death, dike_invest, rfr_cost, evacuation]
    
    robustness_score = robustness_score.append(pd.Series(policy_line, index=["policy", "damage", "death", "dike_invest", "rfr_cost", "evacuation"]), ignore_index=True) 

robustness_new = robustness_score.drop(columns="policy")
epsilon = robustness_new.apply(sp.stats.iqr)
convergence_min = robustness_score.apply(np.min)
convergence_max = robustness_score.apply(np.max)


In [180]:
n_scenarios = 50
scenarios = sample_uncertainties(dike_model, n_scenarios)
epsilons = epsilon.values

convergence = [HyperVolume(convergence_min, convergence_max), EpsilonProgress()]

# Create a filename for saving (and loading if we need)
filename = 'results/MORO_50_nfe10000.pkl'

In [ ]:
BaseEvaluator.reporting_frequency = 0.1

tic = time.time()

with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robust_metrics,
                                                     scenarios=scenarios,
                                                     nfe=10000,
                                                     epsilons=epsilons,
                                                     convergence=convergence,
                                                     convergence_freq=20,
                                                     logging_freq=1
                                                     )

toc = time.time()
print('Total run time:{} min'.format((toc - tic)/60))

# Save the run results
with open(filename, 'wb') as file_pi:
    pickle.dump((results, convergence), file_pi)
